In [1]:
# !pip install http://download.pytorch.org/whl/cu90/torch-1.0.0-cp36-cp36m-linux_x86_64.whl
# !pip install torchvision
# !pip install tqdm

ERROR: torch-1.0.0-cp36-cp36m-linux_x86_64.whl is not a supported wheel on this platform.
  Using cached torchvision-0.12.0-cp38-cp38-macosx_10_9_x86_64.whl (1.3 MB)
  Using cached torch-1.11.0-cp38-none-macosx_10_9_x86_64.whl (129.9 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.10.2
    Uninstalling torch-1.10.2:
      Successfully uninstalled torch-1.10.2


## 손글씨 문자를 판별하는 MLP 작성

In [2]:
import torch
from torch import nn

net = nn.Sequential(
    nn.Linear(64, 32),
    nn.ReLU(),
    nn.Linear(32, 16),
    nn.ReLU(),
    nn.Linear(16, 10)
)

In [3]:
# optimizer 정의
# Loss 함수 정의

from torch import optim
from sklearn.datasets import load_digits
digits = load_digits()

X = digits.data
Y = digits.target

# NumPy의 ndarray를 PyTorch의 Tensor로 변환
X = torch.tensor(X, dtype=torch.float32)
Y = torch.tensor(Y, dtype=torch.int64)

# 소프트맥스 크로스 엔트로피
loss_fn = nn.CrossEntropyLoss()

# Adam
optimizer = optim.Adam(net.parameters())

In [4]:
# 손실 함수의 로그
losses = []

# 100회 반복
for epoc in range(100):
     # backward 메서드로 계산된
    #이전 값을 삭제
    optimizer.zero_grad()

    # 선형 모델로 y의 예측 값 계산
    y_pred = net(X)

    # MSE loss와 w를 사용한 미분 계산
    loss = loss_fn(y_pred, Y)
    loss.backward()

    # 경사를 갱신
    optimizer.step()

    # 수렴 확인을 위해 loss를 기록해둔다
    losses.append(loss.item())